<a href="https://colab.research.google.com/github/Shady-Rouk/CS2-Extra-Credit-Personal-Project/blob/main/Mask_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Extracting train and test data from zipfile

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/'Mask Detector CS 2'/train.zip

In [ ]:
!unzip gdrive/My\ Drive/'Mask Detector CS 2'/test.zip

1. Importing required libraries

In [4]:
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

2. Building the Neural Network

In [5]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

3. Generating data from image

In [ ]:
TRAINING_DIR = "./train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "./test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

4. Initializing a callback checkpoint

In [7]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

5. Training the model

In [8]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/10
132/132 [==============================] - 130s 978ms/step - loss: 0.5604 - acc: 0.7536 - val_loss: 0.1954 - val_acc: 0.9485
Epoch 2/10
132/132 [==============================] - 127s 959ms/step - loss: 0.3056 - acc: 0.8882 - val_loss: 0.2087 - val_acc: 0.9227
Epoch 3/10
132/132 [==============================] - 127s 965ms/step - loss: 0.3168 - acc: 0.8837 - val_loss: 0.1156 - val_acc: 0.9485
Epoch 4/10
132/132 [==============================] - 127s 964ms/step - loss: 0.2580 - acc: 0.8958 - val_loss: 0.0646 - val_acc: 0.9845
Epoch 5/10
132/132 [==============================] - 126s 953ms/step - loss: 0.2112 - acc: 0.9209 - val_loss: 0.0893 - val_acc: 0.9588
Epoch 6/10
132/132 [==============================] - 126s 951ms/step - loss: 0.2300 - acc: 0.9209 - val_loss: 0.1890 - val_acc: 0.8969
Epoch 7/10
132/132 [==============================] - 127s 958ms/step - loss: 0.1990 - acc: 0.9285 - val_loss: 0.0532 - val_acc: 0.9742
Epoch 8/10
132/132 [============================

In [11]:
model.save("./mask_detector_model", save_format='h5')

6. Testing the model

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model("./mask_detector_model")

labels_dict={0:'Mask Off', 1:'Mask On'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
# webcam = cv2.VideoCapture(0) #Use webcam
webcam = cv2.VideoCapture("./testvid.mp4")
# We load the xml file
classifier = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()